In [28]:
# As usual, a bit of setup
import time
import numpy as np
import matplotlib.pyplot as plt
import LOUPE.WILLOW.loupe as lp
import tensorflow as tf
import h5py
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
def temporal_pooling(C3D_Features,mode="max"):
    if mode=="max":
        h = np.max(C3D_Features,axis=0)  
    elif mode=="average":
        h = np.mean(C3D_Features,axis=0) 
    return h


#train_ids = [line.strip() for line in open("id_data/train_ids.csv", 'r')]
filename = "c3d_data/sub_activitynet_v1-3.c3d.hdf5"
video_feature_representation = h5py.File(filename, 'r')
train_ids = ["v_--0edUL8zmA","v_hHiPEAiYKv0","v_u2uoYvo8J5s","v_c_NlYvL96y0","v_sJFgo9H6zNo"]
video_data = dict()
for videoid in train_ids:
    print(videoid)
    proposals_df = pd.read_csv('prop_data/' + videoid + '.csv',sep=' ')
    c3d_features = video_feature_representation[videoid]['c3d_features'].value
    max_frames = c3d_features.shape[0]
    print(max_frames)
    
    for i in range(proposals_df.shape[0]):
        f_init = proposals_df["f-init"][i]
        f_end =  proposals_df["f-end"][i]
        if (f_init < max_frames) and (f_end > 0):
            if f_init < 0:
                f_init = 0
            if f_end > max_frames:
                f_end = max_frames
        
            #print((f_init,f_end))
            #print(c3d_features[f_init:f_end,:].shape)
            h = temporal_pooling(c3d_features[f_init:f_end,:], mode="max")
            if i == 0:
                H = h
            else:
                H = np.column_stack((H,h))
        else:
            proposals_df.drop(proposals_df.index[i])
    video_data[videoid] = H
    

v_--0edUL8zmA
286
v_hHiPEAiYKv0
310
v_u2uoYvo8J5s
780
v_c_NlYvL96y0
468
v_sJFgo9H6zNo
519
